In [11]:
from aocd.models import Puzzle
import itertools
from collections import defaultdict

puzzle = Puzzle(year=2023, day=3)


--- Day 3: Gear Ratios ---

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


In [8]:
def detect_adjacent_symbol(data_grid, positions):
    """Check for all digits positions in a number if there were adjacent symbols."""
    search_ranges = [-1, 0, 1] # Variations of i,j coord to check
    for digit_i, digit_j in positions:
        for i_mod, j_mod in itertools.product(search_ranges, search_ranges):
            search_i = digit_i + i_mod
            search_j = digit_j + j_mod
            try:
                search_token = data_grid[search_i][search_j]
            except IndexError:
                continue
            if not search_token.isdigit() and search_token != ".":  # Check if this adjacent positions contains a symbol.
                return True
    return False  # No adjacent symbol found


def day3a(input_data):
    # Create 2d grid from data
    data_list = input_data.split("\n")
    data_grid = []
    for line in data_list:
        data_grid.append(list(line))
    
    # Init variables
    part_number_sum = 0
    cur_number = {
        "number": "",
        "positions": [],
    }   

    for i, j in itertools.product(range(len(data_grid)), range(len(data_grid[0]))):  # Loop over all positions
        cur_token = data_grid[i][j]
        if cur_token.isdigit():  # Build a number if the positions has a digit
            cur_number["number"] += cur_token
            cur_number["positions"].append((i, j))  # Store locations of all digits
        elif cur_number["number"]:  # If current pos is not a digit and we had a number build up. 
            is_part_number = detect_adjacent_symbol(data_grid, cur_number["positions"])  # Check if there is an adjacent symbol.
            if is_part_number:  
                part_number_sum += int(cur_number["number"])
            cur_number = {  # Reset cur_number
                "number": "",
                "positions": [],
            }
    return part_number_sum


print(puzzle.input_data)
day3a_answer = day3a(puzzle.input_data)

# Submit awnser
print("part number sum:", day3a_answer)
puzzle.answer_a = day3a_answer


.........874.772...........787..........556.....292......141................910............54...............................................
.......*..*.......314............308.......*....*..............156.759.....*................*.......408*954.84..55.......................515
......927.49...........734...............115...738..=....723..........*...599.......+........573.....................324..../508............
........................*.....222..................298....+........313.............504...../.....375.................................114....
.962...262............988..........*170.....................543...........61............827..244.&..................610......310....#.......
...*..........129...............243...................809....&...+.........*...742............%....*96..100.....766...*..308....*......37...
..2...295....../........*336..*......#..185...........*.........502......301...*.......................*...........-.770..-...599...........
.......-.....

gio: https://adventofcode.com/2023/day/3#part2: Operation not supported


In [9]:
%timeit day3a(puzzle.input_data)

2.8 ms ± 65.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


--- Part Two ---

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?


In [14]:
def detect_adjacent_gear_symbol(data_grid, positions):
    search_ranges = [-1, 0, 1] # Variations of i,j coord to check
    for digit_i, digit_j in positions:
        for i_mod, j_mod in itertools.product(search_ranges, search_ranges):
            search_i = digit_i + i_mod
            search_j = digit_j + j_mod
            try:
                search_token = data_grid[search_i][search_j]
            except IndexError:
                continue
            if search_token == "*":  # Check if this adjacent positions contains a gear symbol.
                return search_i, search_j
    return False  # No adjacent gear symbol found


def day3b(input_data):
    # Create 2d grid from data
    data_list = input_data.split("\n")
    data_grid = []
    for line in data_list:
        data_grid.append(list(line))
    
    # Init variables
    cur_number = {
        "number": "",
        "positions": [],
    }
    gear_dict = defaultdict(list) 

    # Gather all numbers that are adjacent to a gear symbol
    for i, j in itertools.product(range(len(data_grid)), range(len(data_grid[0]))):  # Loop over all positions
        cur_token = data_grid[i][j]
        if cur_token.isdigit():  # Build a number if the positions has a digit
            cur_number["number"] += cur_token
            cur_number["positions"].append((i, j))  # Store locations of all digits
        elif cur_number["number"]:  # If current pos is not a digit and we had a number build up. 
            gear_pos = detect_adjacent_gear_symbol(data_grid, cur_number["positions"])  # Check if there is an adjacent symbol.
            if gear_pos:
                gear_dict[gear_pos].append(int(cur_number["number"]))
            cur_number = {  # Reset cur_number
                "number": "",
                "positions": [],
            }
    
    # Find gear symbols with two adjacent numbers
    gear_ratio_sum = 0
    for adjacent_numbers in gear_dict.values():
        if len(adjacent_numbers) == 2:
            gear_ratio_sum += adjacent_numbers[0] * adjacent_numbers[1]

    return gear_ratio_sum


print(puzzle.input_data)
day3b_answer = day3b(puzzle.input_data)

# Submit answer
print("gear ratio sum:", day3b_answer)
puzzle.answer_b = day3b_answer


.........874.772...........787..........556.....292......141................910............54...............................................
.......*..*.......314............308.......*....*..............156.759.....*................*.......408*954.84..55.......................515
......927.49...........734...............115...738..=....723..........*...599.......+........573.....................324..../508............
........................*.....222..................298....+........313.............504...../.....375.................................114....
.962...262............988..........*170.....................543...........61............827..244.&..................610......310....#.......
...*..........129...............243...................809....&...+.........*...742............%....*96..100.....766...*..308....*......37...
..2...295....../........*336..*......#..185...........*.........502......301...*.......................*...........-.770..-...599...........
.......-.....

gio: https://adventofcode.com/2023/day/3#part2: Operation not supported


In [15]:
%timeit day3b(puzzle.input_data)

3.16 ms ± 29.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
